<a href="https://colab.research.google.com/github/ereinha/SineKAN/blob/main/FourierKAN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from fourier_kan import *

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

from sklearn.metrics import precision_score, recall_score, f1_score

In [5]:
# Load MNIST
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)
trainset = torchvision.datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)
valset = torchvision.datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)
train_loader = DataLoader(trainset, batch_size=64, num_workers=2, shuffle=True)
val_loader = DataLoader(valset, batch_size=64, num_workers=2, shuffle=False)

In [ ]:
epochs = 30
lrs = [1e-4, 2e-4, 3e-4, ]
gammas = [0.8, 0.9]
hdims = [64, 128, 256]
best_accs = []
for lr in lrs:
    for gamma in gammas:
        for hdim in hdims:
            torch.manual_seed(42)
            best_acc = 0
            # Define model
            model = FourierKAN(layers_hidden=[28 * 28, hdim, 10], grid_size=8)
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model.to(device)
            # Define optimizer
            optimizer = optim.AdamW(model.parameters(), lr=lr)
            # Define learning rate scheduler
            scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)

            # Define loss
            criterion = nn.CrossEntropyLoss()
            for epoch in range(epochs):
                # Train
                model.train()
                with tqdm(train_loader) as pbar:
                    for i, (images, labels) in enumerate(pbar):
                        images = images.view(-1, 28 * 28).to(device)
                        optimizer.zero_grad()
                        output = model(images)
                        loss = criterion(output, labels.to(device))
                        loss.backward()
                        optimizer.step()
                        accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()
                        pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])

                # Validation
                model.eval()
                val_loss = 0
                val_accuracy = 0
                with torch.no_grad():
                    for images, labels in val_loader:
                        images = images.view(-1, 28 * 28).to(device)
                        output = model(images)
                        val_loss += criterion(output, labels.to(device)).item()
                        val_accuracy += (
                            (output.argmax(dim=1) == labels.to(device)).float().mean().item()
                        )
                val_loss /= len(val_loader)
                val_accuracy /= len(val_loader)
                if val_accuracy > best_acc:
                    best_acc = val_accuracy

                # Update learning rate
                scheduler.step()

                print(
                    f"Epoch {epoch + 1}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}"
                )
            best_accs.append(best_acc)
            print(f"LR: {lr} Gamma: {gamma} Hdim: {hdim} Best Accuracy: {best_acc}")

100%|██████████| 938/938 [00:26<00:00, 35.34it/s, accuracy=0.719, loss=1.3, lr=0.0001]


Epoch 1, Val Loss: 1.0968678582246136, Val Accuracy: 0.6793391719745223


100%|██████████| 938/938 [00:20<00:00, 46.09it/s, accuracy=0.75, loss=0.612, lr=8e-5]


Epoch 2, Val Loss: 0.5633355475914706, Val Accuracy: 0.8553941082802548


100%|██████████| 938/938 [00:21<00:00, 44.37it/s, accuracy=0.875, loss=0.448, lr=6.4e-5]


...

Epoch 28, Val Loss: 0.16144642319918054, Val Accuracy: 0.9543192675159236


100%|██████████| 938/938 [00:30<00:00, 30.54it/s, accuracy=1, loss=0.000406, lr=1.57e-5]


Epoch 29, Val Loss: 0.1606314708404599, Val Accuracy: 0.9541202229299363


100%|██████████| 938/938 [00:29<00:00, 32.10it/s, accuracy=1, loss=0.00289, lr=1.41e-5]


Epoch 30, Val Loss: 0.1603144183387879, Val Accuracy: 0.9552149681528662
LR: 0.0003 Gamma: 0.9 Hdim: 256 Best Accuracy: 0.9552149681528662


In [ ]:
epochs = 30
wds = [0.01, 0.1, 0.25, 0.5, 0.75, 1.]
wd_accs = np.empty((6, epochs))
for h, wd in enumerate(wds):
    torch.manual_seed(42)
    best_acc = 0
    # Define model
    model = FourierKAN(layers_hidden=[28 * 28, 64, 10], grid_size=8)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # Define optimizer
    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=wd)
    # Define learning rate scheduler
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    # Define loss
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        # Train
        model.train()
        with tqdm(train_loader) as pbar:
            for i, (images, labels) in enumerate(pbar):
                images = images.view(-1, 28 * 28).to(device)
                optimizer.zero_grad()
                output = model(images)
                loss = criterion(output, labels.to(device))
                loss.backward()
                optimizer.step()
                accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()
                pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])

        # Validation
        model.eval()
        val_loss = 0
        val_accuracy = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.view(-1, 28 * 28).to(device)
                output = model(images)
                val_loss += criterion(output, labels.to(device)).item()
                val_accuracy += (
                    (output.argmax(dim=1) == labels.to(device)).float().mean().item()
                )

        val_loss /= len(val_loader)
        val_accuracy /= len(val_loader)
        wd_accs[h, epoch] = val_accuracy

        # Update learning rate
        scheduler.step()

        print(
            f"Epoch {epoch + 1}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}"
        )

100%|██████████| 938/938 [00:23<00:00, 40.48it/s, accuracy=0.719, loss=1.3, lr=0.0001]


Epoch 1, Val Loss: 1.0968678582246136, Val Accuracy: 0.6793391719745223


100%|██████████| 938/938 [00:20<00:00, 46.74it/s, accuracy=0.875, loss=0.598, lr=9e-5]


Epoch 2, Val Loss: 0.5249082095873584, Val Accuracy: 0.8628582802547771


100%|██████████| 938/938 [00:20<00:00, 44.92it/s, accuracy=0.875, loss=0.422, lr=8.1e-5]


...

Epoch 28, Val Loss: 0.13293363324479218, Val Accuracy: 0.9598925159235668


100%|██████████| 938/938 [00:21<00:00, 44.50it/s, accuracy=1, loss=0.00175, lr=5.23e-6]


Epoch 29, Val Loss: 0.13126693130359587, Val Accuracy: 0.9591958598726115


100%|██████████| 938/938 [00:20<00:00, 45.13it/s, accuracy=1, loss=0.00502, lr=4.71e-6]


Epoch 30, Val Loss: 0.1315764596335038, Val Accuracy: 0.9596934713375797


In [ ]:
epochs = 30
hdims = [16, 32, 64, 128, 256, 512]
hdim_metrics = {
    'accuracy': np.empty((6, epochs)),
    'precision': np.empty((6, epochs)),
    'recall': np.empty((6, epochs)),
    'f1_score': np.empty((6, epochs)),
}
for h, hdim in enumerate(hdims):
    torch.manual_seed(42)
    # Define model
    model = FourierKAN(layers_hidden=[28 * 28, hdim, 10], grid_size=8)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # Define optimizer
    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1.)
    # Define learning rate scheduler
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    # Define loss
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        # Train
        model.train()
        with tqdm(train_loader) as pbar:
            for i, (images, labels) in enumerate(pbar):
                images = images.view(-1, 28 * 28).to(device)
                optimizer.zero_grad()
                output = model(images)
                loss = criterion(output, labels.to(device))
                loss.backward()
                optimizer.step()
                accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()
                pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])

        # Validation
        model.eval()
        all_preds = []
        all_labels = []
        val_loss = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.view(-1, 28 * 28).to(device)
                output = model(images)
                val_loss += criterion(output, labels.to(device)).item()
                preds = output.argmax(dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss /= len(val_loader)
        val_accuracy = (np.array(all_preds) == np.array(all_labels)).mean()
        val_precision = precision_score(all_labels, all_preds, average='weighted')
        val_recall = recall_score(all_labels, all_preds, average='weighted')
        val_f1 = f1_score(all_labels, all_preds, average='weighted')

        # Store metrics
        hdim_metrics['accuracy'][h, epoch] = val_accuracy
        hdim_metrics['precision'][h, epoch] = val_precision
        hdim_metrics['recall'][h, epoch] = val_recall
        hdim_metrics['f1_score'][h, epoch] = val_f1

        # Update learning rate
        scheduler.step()

        print(
            f"Epoch {epoch + 1}, Val Loss: {val_loss}, "
            f"Val Accuracy: {val_accuracy}, Precision: {val_precision}, "
            f"Recall: {val_recall}, F1 Score: {val_f1}"
        )

100%|██████████| 938/938 [00:27<00:00, 33.86it/s, accuracy=0.406, loss=1.6, lr=0.0001]


Epoch 1, Val Loss: 1.5846304597368666, Val Accuracy: 0.4925, Precision: 0.4947126704344288, Recall: 0.4925, F1 Score: 0.4750808815087283


100%|██████████| 938/938 [00:34<00:00, 27.45it/s, accuracy=0.75, loss=0.948, lr=9e-5]


Epoch 2, Val Loss: 0.8578524407307813, Val Accuracy: 0.8133, Precision: 0.8135013271503468, Recall: 0.8133, F1 Score: 0.8124719084741753


100%|██████████| 938/938 [00:24<00:00, 38.88it/s, accuracy=0.875, loss=0.438, lr=8.1e-5]


...

Epoch 28, Val Loss: 0.09936974312125704, Val Accuracy: 0.9714, Precision: 0.9713901777154038, Recall: 0.9714, F1 Score: 0.9713722488945806


100%|██████████| 938/938 [00:45<00:00, 20.84it/s, accuracy=1, loss=0.000213, lr=5.23e-6]


Epoch 29, Val Loss: 0.10011130917964228, Val Accuracy: 0.9719, Precision: 0.9719106325296081, Recall: 0.9719, F1 Score: 0.9718829772247319


100%|██████████| 938/938 [00:44<00:00, 20.84it/s, accuracy=1, loss=0.000128, lr=4.71e-6]


Epoch 30, Val Loss: 0.10226487026967293, Val Accuracy: 0.9716, Precision: 0.9716259786081478, Recall: 0.9716, F1 Score: 0.9715823446823495


In [ ]:
np.save('./saved_results/fourier_hdim_metrics.npy', hdim_metrics)

In [ ]:
epochs = 30
n_layers = [1, 2, 3, 4]
nlayer_metrics = {
    'accuracy': np.empty((4, epochs)),
    'precision': np.empty((4, epochs)),
    'recall': np.empty((4, epochs)),
    'f1_score': np.empty((4, epochs)),
}
for h, n_layer in enumerate(n_layers):
    torch.manual_seed(42)
    # Define model
    model = FourierKAN(layers_hidden=[28 * 28] + [128]*n_layer + [10], grid_size=8)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # Define optimizer
    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1.)
    # Define learning rate scheduler
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    # Define loss
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        # Train
        model.train()
        with tqdm(train_loader) as pbar:
            for i, (images, labels) in enumerate(pbar):
                images = images.view(-1, 28 * 28).to(device)
                optimizer.zero_grad()
                output = model(images)
                loss = criterion(output, labels.to(device))
                loss.backward()
                optimizer.step()
                accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()
                pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])

        # Validation
        model.eval()
        all_preds = []
        all_labels = []
        val_loss = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.view(-1, 28 * 28).to(device)
                output = model(images)
                val_loss += criterion(output, labels.to(device)).item()
                preds = output.argmax(dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss /= len(val_loader)
        val_accuracy = (np.array(all_preds) == np.array(all_labels)).mean()
        val_precision = precision_score(all_labels, all_preds, average='weighted')
        val_recall = recall_score(all_labels, all_preds, average='weighted')
        val_f1 = f1_score(all_labels, all_preds, average='weighted')

        # Store metrics
        nlayer_metrics['accuracy'][h, epoch] = val_accuracy
        nlayer_metrics['precision'][h, epoch] = val_precision
        nlayer_metrics['recall'][h, epoch] = val_recall
        nlayer_metrics['f1_score'][h, epoch] = val_f1

        # Update learning rate
        scheduler.step()

        print(
            f"Epoch {epoch + 1}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}"
        )

100%|██████████| 938/938 [00:28<00:00, 32.99it/s, accuracy=0.844, loss=0.668, lr=0.0001]


Epoch 1, Val Loss: 0.7063552182950791, Val Accuracy: 0.8087


100%|██████████| 938/938 [00:22<00:00, 42.25it/s, accuracy=0.938, loss=0.322, lr=9e-5]


Epoch 2, Val Loss: 0.4250892712052461, Val Accuracy: 0.884


100%|██████████| 938/938 [00:23<00:00, 40.66it/s, accuracy=0.844, loss=0.447, lr=8.1e-5]


...

Epoch 28, Val Loss: 0.949599187085583, Val Accuracy: 0.662


100%|██████████| 938/938 [00:23<00:00, 39.10it/s, accuracy=0.656, loss=1.06, lr=5.23e-6]


Epoch 29, Val Loss: 0.9043705379886992, Val Accuracy: 0.6903


100%|██████████| 938/938 [00:25<00:00, 36.95it/s, accuracy=0.719, loss=0.668, lr=4.71e-6]


Epoch 30, Val Loss: 0.8524398758153248, Val Accuracy: 0.718


In [ ]:
np.save('./saved_results/fourier_layer_metrics.npy', nlayer_metrics)

In [6]:
from torch.profiler import profile, record_function, ProfilerActivity
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
def profile_model(model, inputs, num_runs=100):
    total_times = []
    for _ in range(num_runs):
        total_time = 0
        with profile(activities=[ProfilerActivity.CUDA], record_shapes=True) as prof:
            model(inputs)

        # Extract total CUDA time spent on 'model_inference'
        for event in prof.key_averages():
            total_time += event.cuda_time

        total_times.append(total_time)
    average_time = np.mean(total_times)
    std_time = np.std(total_times)
    print(f"Average inference time: {average_time} microseconds +/- {std_time}")
    return average_time, std_time

In [8]:
inputs = torch.randn((1, 784)).to(device)
hdims = [16, 32, 64, 128, 256, 512]
means = []
stds = []
for h in hdims:
    model = FourierKAN(layers_hidden=[28 * 28, h, 10], grid_size=8).to(device)
    print('Hidden Dims: %d' % h)
    avg, std = profile_model(model, inputs, 1000)
    means.append(avg)
    stds.append(std)

Hidden Dims: 16


<ipython-input-7-d37aa78eb809>:10: FutureWarning: `cuda_time` is deprecated, please use `device_time` instead.
  total_time += event.cuda_time


Average inference time: 47.790049749999994 microseconds +/- 0.1392367909800007
Hidden Dims: 32
Average inference time: 50.335594249999986 microseconds +/- 0.11134498385395292
Hidden Dims: 64
Average inference time: 56.19825599999999 microseconds +/- 0.11354726918336075
Hidden Dims: 128
Average inference time: 69.25129575000003 microseconds +/- 0.1910046975062024
Hidden Dims: 256
Average inference time: 88.30248124999999 microseconds +/- 0.24689817554194018
Hidden Dims: 512
Average inference time: 126.77820150000001 microseconds +/- 0.1522947771519121


In [9]:
np.save('./saved_results/fourier_scaling_mean_hdims.npy', means)
np.save('./saved_results/fourier_scaling_std_hdims.npy', stds)

In [10]:
hdims = [16, 32, 64, 128, 256, 512]
means = []
stds = []
for h in hdims:
    model = FourierKAN(layers_hidden=[28 * 28, 64, 10], grid_size=8).to(device)
    inputs = torch.randn((h, 784)).to(device)
    print('Input Batch Size: %d' % h)
    avg, std = profile_model(model, inputs, 1000)
    means.append(avg)
    stds.append(std)

Input Batch Size: 16


<ipython-input-7-d37aa78eb809>:10: FutureWarning: `cuda_time` is deprecated, please use `device_time` instead.
  total_time += event.cuda_time


Average inference time: 190.92044850000002 microseconds +/- 0.14409671048552772
Input Batch Size: 32
Average inference time: 342.97200300000003 microseconds +/- 0.22171968731035682
Input Batch Size: 64
Average inference time: 628.6962120000001 microseconds +/- 11.176362440959107
Input Batch Size: 128
Average inference time: 1164.56823575 microseconds +/- 35.10362208072135
Input Batch Size: 256
Average inference time: 2352.4976312500003 microseconds +/- 2.9762258931759447
Input Batch Size: 512
Average inference time: 4717.754778750001 microseconds +/- 4.4123682091634375


In [11]:
np.save('./saved_results/fourier_scaling_mean_batch.npy', means)
np.save('./saved_results/fourier_scaling_std_batch.npy', stds)

In [12]:
nlayers = [1, 2, 3, 4]
means = []
stds = []
for n in nlayers:
    model = FourierKAN(layers_hidden=[28 * 28] + [64]*n +[10], grid_size=8).to(device)
    inputs = torch.randn((1, 784)).to(device)
    print('Number of Hidden Layers: %d' % n)
    avg, std = profile_model(model, inputs, 1000)
    means.append(avg)
    stds.append(std)

Number of Hidden Layers: 1


<ipython-input-7-d37aa78eb809>:10: FutureWarning: `cuda_time` is deprecated, please use `device_time` instead.
  total_time += event.cuda_time


Average inference time: 50.2811605 microseconds +/- 11.008528536479783
Number of Hidden Layers: 2
Average inference time: 49.61591166666667 microseconds +/- 0.07979245527895706
Number of Hidden Layers: 3
Average inference time: 46.85499637499999 microseconds +/- 0.0530537375213529
Number of Hidden Layers: 4
Average inference time: 45.1002498 microseconds +/- 0.05149992873742776


In [13]:
np.save('./saved_results/fourier_scaling_mean_layers.npy', means)
np.save('./saved_results/fourier_scaling_std_layers.npy', stds)